In [1]:
from sklearn import datasets
breast_cancer=datasets.load_breast_cancer()
x,y=breast_cancer.data,breast_cancer.target

In [2]:
n_samples,n_features=x.shape
print(n_samples)
print(n_features)

569
30


In [3]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test= train_test_split(x,y,test_size=0.2)

In [4]:
# pre processing
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)


In [5]:
import numpy as np
# import pytorch and torch.nn and convert to tensor
import torch
import torch.nn as nn
import torch.nn.functional as F
x_train = torch.from_numpy(x_train).float()
y_train = torch.from_numpy(y_train).float()
x_test = torch.from_numpy(x_test).float()
y_test = torch.from_numpy(y_test).float()


In [6]:
y_train=y_train.view(y_train.shape[0],1)
y_test=y_test.view(y_test.shape[0],1)

In [7]:
# building logistic regression model 
class logistic_reg(torch.nn.Module):
    def __init__(self,no_of_input):
        super(logistic_reg,self).__init__()
        self.layer1=torch.nn.Linear(no_of_input,20)
        self.layer2=torch.nn.Linear(20,1)
    def forward(self,x):
        y_predicted = self.layer1(x)
        y_predicted= torch.sigmoid(self.layer2(y_predicted))
        return y_predicted

In [8]:
# training and optimization
model=logistic_reg(n_features)

In [9]:
criterion=torch.nn.BCELoss()
optimizer=torch.optim.SGD(model.parameters(),lr=0.01)

In [10]:
number_of_epochs=100
for epoch in range(number_of_epochs):
 y_prediction=model(x_train)
 loss=criterion(y_prediction,y_train)
 loss.backward()
 optimizer.step()
 optimizer.zero_grad()
 if (epoch+1)%10 == 0:
   print('epoch:', epoch+1,',loss=',loss.item())

epoch: 10 ,loss= 0.6508088707923889
epoch: 20 ,loss= 0.5610125064849854
epoch: 30 ,loss= 0.4952756464481354
epoch: 40 ,loss= 0.44450584053993225
epoch: 50 ,loss= 0.4038459062576294
epoch: 60 ,loss= 0.37046805024147034
epoch: 70 ,loss= 0.34256815910339355
epoch: 80 ,loss= 0.31891170144081116
epoch: 90 ,loss= 0.29861220717430115
epoch: 100 ,loss= 0.28101247549057007


In [11]:
with torch.no_grad():
 y_pred=model(x_test)
 y_pred_class=y_pred.round()
 accuracy=(y_pred_class.eq(y_test).sum())/float(y_test.shape[0])
 print(accuracy.item())

0.9649122953414917


In [15]:
# training loop to calculate loss
number_of_epochs=300
for epoch in range(number_of_epochs):
    running_loss=0
    for i,data in enumerate(zip(x_train,y_train)):
        inputs,labels=data
        optimizer.zero_grad()
        outputs=model(inputs)
        loss=criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        running_loss+=loss.item()

    if ((epoch%50)==0):
        print('epoch:',epoch+1,'i:',i+1,'loss:',running_loss/300)

epoch: 1 i: 455 loss: 0.044667169780919475
epoch: 51 i: 455 loss: 0.04231144706098699
epoch: 101 i: 455 loss: 0.04029132976350813
epoch: 151 i: 455 loss: 0.03850677359895083
epoch: 201 i: 455 loss: 0.03684166415994923
epoch: 251 i: 455 loss: 0.035184245136834796


In [18]:
# calculate inference time and loss
start_time = time.time()
plain_pred=model(x_test)
end_time = time.time()

In [21]:
print('inference time:',(time.time()-start_time)/60)
print('loss:',criterion(plain_pred,y_test).item())

inference time: 0.6603620767593383
loss: 1.0402671098709106
